# Avaliação Final I - Visualização de dados

## Autor: José Norberto Fagundes Isaias (19202785)

In [87]:
import pandas as pd
import altair as alt

### Fase 1 - preparação dos dados

1. Carregamento dos datasets em .csv e criação de DataFrames contendo apenas as colunas desejadas.
2. Manipulação dos DFs para agrupar os dados por ano.
3. União das duas tabelas através do nome e ano de lançamento dos jogos.
4. Criação de um DataFrame com ano, a quantidade de vendas de um gênero (do dataset de dados do IMDB) naquele ano, a quantidade de jogos lançados daquele gênero no ano e o gênero em si.
5. Declaraçao de um DF com a nota média por gênero (do dataset de vendas).
6. Criação de um DF específico para o uso no gráfico com os dados de vendas por gênero.

In [88]:
# 1
raw_sales = pd.read_csv("Datasets/video games sales.csv")
raw_imdb = pd.read_csv("Datasets/imdb-videogames.csv")

df_sales = raw_sales[["Name", "Year", "Genre", "Global_Sales"]].dropna()
df_imdb = raw_imdb[["name", "year", "rating", "Action", "Adventure", "Comedy", "Crime", "Family", "Mystery", "Sci-Fi", "Thriller"]].dropna()

df_imdb.rename(columns={"year": "Year", "name": "Name"}, inplace=True)

# 2
df_quant_gen = df_imdb.groupby("Year")[["Action", "Adventure", "Comedy", "Crime", "Family", "Mystery", "Sci-Fi", "Thriller"]].sum().reset_index()

# 3
merged = pd.merge(df_sales, df_imdb, on=["Name", "Year"], how="inner")

# 4
genres = ['Action', 'Adventure', 'Comedy', 'Crime', 'Family', 'Mystery', 'Sci-Fi', 'Thriller']
df_sales_quant_gen = []
for g in genres:
    df_temp = merged.groupby("Year")[["Global_Sales", g]].sum().reset_index()
    df_temp = pd.DataFrame({
                    'Year': df_temp["Year"],
                    'Global_Sales': df_temp["Global_Sales"],
                    'Quant_gen': df_quant_gen[g].loc[df_quant_gen["Year"].isin(df_temp["Year"])].values,
                    'Genero': g
                })
    df_sales_quant_gen.append(df_temp)

df_sales_quant = pd.concat(df_sales_quant_gen, ignore_index=True)

# 5
df_ratings = merged.groupby("Genre")["rating"].mean().reset_index()

# 6
dfs = []
for g in genres:
    df_temp = pd.DataFrame({
        'x': df_sales_quant['Year'],
        'y': (df_sales_quant['Global_Sales'] / df_sales_quant["Quant_gen"]),
        'Gênero': df_sales_quant['Genero']
    })
    dfs.append(df_temp)
df_line = pd.concat(dfs, ignore_index=True)

# Tentativa de preparar os dados para criar um gráfico com múltiplas métricas (quantidade de jogos, vendas totais e vendas médias por gênero)

# df_count = merged_line.melt(
#     id_vars=['Year'],
#     value_vars=genres,
#     var_name='Genre',
#     value_name='Value'
# )
# df_count['Metric'] = 'Quant. Jogos'

# df_sales = pd.DataFrame({
#     'Year': merged_line['Year'].repeat(len(genres)),
#     'Genre': genres * len(merged_line),
#     'Value': merged_line['Global_Sales'].repeat(len(genres)),
#     'Metric': 'Sales'
# })

# dfs = []
# for g in genres:
#     df_temp = pd.DataFrame({
#         'Year': merged_line['Year'],
#         'Genre': g,
#         'Value': merged_line['Global_Sales'] / merged_line[g],
#         'Metric': 'Avg_per_game'
#     })
#     dfs.append(df_temp)

# df_avg = pd.concat(dfs, ignore_index=True)

# for g in genres:
#     df_line = pd.DataFrame({
#         'x': merged_line['Year'],
#         'y': merged_line['Global_Sales'] / merged_line[g],
#         'serie': g

#     })

#df_long = pd.concat([df_count, df_sales, df_avg], ignore_index=True)

# df_line.head(20)


### Fase 2 - configuração dos gráficos

1. Criação de parâmetro para a barra deslizante que controla o ano de início do gráfico de vendas por gênero
2. Instanciação do gráfico de Vendas Globais em milhões divido pela quantidade de jogos de cada gênero (eixo y) ao longo dos anos (eixo x). Aqui será criado um gráfico de linhas com pontos com uma série para cada gênero.
3. Instanciação do gráfico barras de média de nota no IMDB por gênero.

In [89]:
# 1
year_min = int(df_line['x'].min())
year_max = int(df_line['x'].max())
year_param = alt.param(name='Ano', bind=alt.binding_range(min=year_min, max=year_max, step=1), value=2000)

# type_dropdown = alt.param(
#     name='MetricaEscolhida',
#     bind=alt.binding_select(options=df_long['Metric'].unique().tolist()),
#     value='Avg_per_game'   # valor inicial
# )

# 2
graph_sales = alt.Chart(df_line).mark_line(point=True).encode(
    x=alt.X('x:O', title='Ano'),
    y=alt.Y('y:Q', title='Vendas globais (M) / quantidade de jogos'),
    color=alt.Color('Gênero:N', title='Gênero'),
    tooltip=[alt.Tooltip('x:O', title='Ano'), alt.Tooltip('y:Q', title='Vendas globais (M) / quantidade de jogos'), alt.Tooltip('Gênero:N', title='Gênero')]
).properties(
    title='Relação entre Vendas Globais e Quantidade de Jogos por Gênero ao Longo dos Anos'
).transform_filter(
    alt.datum.x >= year_param,
    #alt.datum.Metric == type_dropdown
).add_params(year_param).properties(width=600, height=350)

# 3
graph_rating = alt.Chart(df_ratings).mark_bar().encode(
    x=alt.X("Genre:O", title="Genre"),
    y=alt.Y("rating:Q", title="Average Rating"),
    color=alt.Color("Genre:N", legend=None),
    tooltip=[alt.Tooltip('Genre:O', title='Gênero'), alt.Tooltip('rating:Q', title='Média de Avaliação')]
).properties(
    width = 500,
    title = "Nota média por gênero de jogo"
)

### Fase 3 - chamada dos gráficos

Aqui são desenhados os gráficos criados anteriormente.

In [90]:
graph_sales.interactive()

alt.Chart(...)

In [91]:
graph_rating.interactive()

alt.Chart(...)